# Adding Memory to a Chain (Part 2): Creating the Chain

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.runnables import chain

from langchain_openai import ChatOpenAI

from langchain.memory import ConversationSummaryMemory

from operator import itemgetter

In [3]:
TEMPLATE = '''
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{message_log}

Human: 
{question}

AI:
'''

prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [4]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

/opt/anaconda3/envs/langchain_env_benji/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3519: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [5]:
chat_memory = ConversationSummaryMemory(llm = ChatOpenAI(),
                                        memory_key="message_log")

/var/folders/l0/l8bzvy196wn95cjygzkqtsfw0000gn/T/ipykernel_2119/3002585002.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_memory = ConversationSummaryMemory(llm = ChatOpenAI(),


In [6]:
question = "Can you give me an interesting fact I probably didn't know about?"
# question = "Can you elaborate a bit more on this fact?"

In [7]:
dictionary_output = RunnablePassthrough.assign(
    message_log = RunnableLambda(chat_memory.load_memory_variables) | 
    itemgetter('message_log')).invoke(
    {'question':question})

In [8]:
prompt_value_output = prompt_template.invoke(dictionary_output)

In [9]:
ai_message_output = chat.invoke(prompt_value_output)

In [10]:
response = StrOutputParser().invoke(ai_message_output)

In [11]:
chat_memory.save_context(inputs = {'input':question}, 
                         outputs = {'output':response})

In [12]:
chat_memory.load_memory_variables({})

{'message_log': "The human asks for an interesting fact they probably didn't know about. The AI shares that octopuses have three hearts and explains how they function."}

In [13]:
chain1 = (
    RunnablePassthrough.assign(
        message_log = RunnableLambda(chat_memory.load_memory_variables) | 
        itemgetter('message_log')) 
    | prompt_template 
    | chat 
    | StrOutputParser()
)

question = "Can you elaborate a bit more on this fact?"

response = chain1.invoke({'question':question})

chat_memory.save_context(inputs = {'input':question}, 
                         outputs = {'output':response})

response

"Absolutely! Octopuses have three hearts, two of which work exclusively to move blood beyond the animal's gills, while the third keeps circulation flowing for the organs. The third heart actually stops beating when the octopus swims, explaining the species' penchant for crawling rather than swimming, which exhausts them. Furthermore, the blood of an octopus is blue rather than red. This is because their blood uses copper-rich hemocyanin for oxygen transport rather than the iron-rich hemoglobin that humans"

In [14]:
@chain
def memory_chain(question):
    
    chain1 = (
        RunnablePassthrough.assign(
            message_log = RunnableLambda(chat_memory.load_memory_variables) | 
            itemgetter('message_log')) 
        | prompt_template 
        | chat 
        | StrOutputParser()
    )
    
    chain1.get_graph().print_ascii()

    response = chain1.invoke({'question':question})

    chat_memory.save_context(inputs = {'input':question}, 
                             outputs = {'output':response})

    return response

In [15]:
memory_chain.invoke("Can you elaborate a bit more on this fact?")

PydanticUserError: `load_memory_variables_input` is not fully defined; you should define `Dict`, then call `load_memory_variables_input.model_rebuild()`.

For further information visit https://errors.pydantic.dev/2.11/u/class-not-fully-defined